In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 上证指数日收盘价
df_index = pd.read_excel(r'C:\Users\62589\Desktop\IDX_Idxtrd.xlsx')

In [3]:
# i.计算日收益率
df_index['return'] = df_index['Idxtrd05'].pct_change()
df_index

,Indexcd,Idxtrd01,Idxtrd05,return
0,300,2022-11-01,3634.173,NaN
1,300,2022-11-02,3677.805,0.012006
2,300,2022-11-03,3647.897,-0.008132
3,300,2022-11-04,3767.175,0.032698
4,300,2022-11-07,3775.297,0.002156
...,...,...,...,...
238,300,2023-10-25,3504.458,0.004969
239,300,2023-10-26,3514.145,0.002764
240,300,2023-10-27,3562.385,0.013727
241,300,2023-10-30,3583.772,0.006004


In [4]:
# 读取所有个股数据（2个文件进行拼接）
df_stock1 = pd.read_excel(r'C:\Users\62589\Desktop\TRD_Dalyr.xlsx')
df_stock2 = pd.read_excel(r'C:\Users\62589\Desktop\TRD_Dalyr1.xlsx')

In [5]:
df_stock = df_stock1.append(df_stock2,ignore_index=True)
df_stock

,Stkcd,Trddt,Clsprc
0,1,2022-11-01,10.670
1,1,2022-11-02,10.630
2,1,2022-11-03,10.440
3,1,2022-11-04,10.820
4,1,2022-11-07,10.880
...,...,...,...
1271139,900957,2023-10-25,0.356
1271140,900957,2023-10-26,0.364
1271141,900957,2023-10-27,0.370
1271142,900957,2023-10-30,0.393


In [6]:
# ii.计算所有上市公司的日收益率
df_stock['return'] = ''
for i in range(len(df_stock)-1):
    if df_stock.loc[i+1,'Stkcd'] == df_stock.loc[i,'Stkcd']:
        df_stock.loc[i+1,'return'] = df_stock.loc[i+1,'Clsprc'] / df_stock.loc[i,'Clsprc'] - 1
    else:
        pass

In [7]:
df_stock

,Stkcd,Trddt,Clsprc,return
0,1,2022-11-01,10.670,
1,1,2022-11-02,10.630,-0.003749
2,1,2022-11-03,10.440,-0.017874
3,1,2022-11-04,10.820,0.036398
4,1,2022-11-07,10.880,0.005545
...,...,...,...,...
1271139,900957,2023-10-25,0.356,0.017143
1271140,900957,2023-10-26,0.364,0.022472
1271141,900957,2023-10-27,0.370,0.016484
1271142,900957,2023-10-30,0.393,0.062162


In [8]:
# 获取每股净资产数据（只选A股）
net = pd.read_excel(r'C:\Users\62589\Desktop\FI_T9.xlsx')
net = net.loc[net['Typrep']=='A']
net

,Stkcd,ShortName,Accper,Typrep,F091001A
0,1,平安银行,2022-12-31,A,22.399258
2,1,平安银行,2023-03-31,A,23.020973
4,1,平安银行,2023-06-30,A,23.295527
6,1,平安银行,2023-09-30,A,24.005978
8,2,万科A,2022-12-31,A,34.820877
...,...,...,...,...,...
34181,900953,凯马B,2023-09-30,A,0.894316
34183,900957,凌云B股,2022-12-31,A,1.624429
34185,900957,凌云B股,2023-03-31,A,1.649415
34187,900957,凌云B股,2023-06-30,A,1.679721


In [9]:
# iii.计算每日账面市值比
df_stock['PB'] = ''
for i in range(len(df_stock)):
    df_pick = net.loc[(net['Stkcd']==df_stock.loc[i,'Stkcd'])&(net['Accper']>=df_stock.loc[i,'Trddt'])]
    try:
        if len(df_pick)>0:
            df_stock.loc[i,'PB'] = df_stock.loc[i,'Clsprc'] / df_pick['F091001A'].tolist()[0]
        else:
            df_pick = net.loc[(net['Stkcd']==df_stock.loc[i,'Stkcd'])]
            df_stock.loc[i,'PB'] = df_stock.loc[i,'Clsprc'] / df_pick['F091001A'].tolist()[-1]
    except:
        pass

In [10]:
df_stock

,Stkcd,Trddt,Clsprc,return,PB
0,1,2022-11-01,10.670,,0.476355
1,1,2022-11-02,10.630,-0.003749,0.474569
2,1,2022-11-03,10.440,-0.017874,0.466087
3,1,2022-11-04,10.820,0.036398,0.483052
4,1,2022-11-07,10.880,0.005545,0.48573
...,...,...,...,...,...
1271139,900957,2023-10-25,0.356,0.017143,0.207639
1271140,900957,2023-10-26,0.364,0.022472,0.212305
1271141,900957,2023-10-27,0.370,0.016484,0.215804
1271142,900957,2023-10-30,0.393,0.062162,0.229219


In [11]:
# 获取每股总市值数据（只选A股）
mv = pd.read_excel(r'C:\Users\62589\Desktop\FI_T10.xlsx')
mv

,Stkcd,ShortName,Accper,F100801A
0,1,平安银行,2022-12-31,5.142219e+12
1,1,平安银行,2023-03-31,5.252312e+12
2,1,平安银行,2023-06-30,5.266384e+12
3,1,平安银行,2023-09-30,5.267879e+12
4,2,万科A,2022-12-31,1.595625e+12
...,...,...,...,...
20704,900953,凯马B,2023-09-30,4.005438e+09
20705,900957,凌云B股,2022-12-31,1.490310e+09
20706,900957,凌云B股,2023-03-31,1.432902e+09
20707,900957,凌云B股,2023-06-30,1.412802e+09


In [12]:
# iv.合并dataframe
agg_df = df_stock.copy()
agg_df['Maeket Value'] = ''
for i in range(len(agg_df)):
    df_pick = mv.loc[(mv['Stkcd']==agg_df.loc[i,'Stkcd'])&(mv['Accper']>=agg_df.loc[i,'Trddt'])]
    try:
        if len(df_pick)>0:
            agg_df.loc[i,'Maeket Value'] = mv['F100801A'].tolist()[0]
        else:
            df_pick = mv.loc[(mv['Stkcd']==agg_df.loc[i,'Stkcd'])]
            agg_df.loc[i,'Maeket Value'] = mv['F100801A'].tolist()[-1]
    except:
        pass

In [13]:
# 获取行业数据
industry = pd.read_excel(r'C:\Users\62589\Desktop\STK_LISTEDCOINFOANL.xlsx')
industry

,Symbol,ShortName,EndDate,IndustryName
0,1,平安银行,2022-12-31,货币金融服务
1,2,万科A,2022-12-31,房地产业
2,4,ST 国华,2022-12-31,软件和信息技术服务业
3,5,ST 星源,2022-12-31,生态保护和环境治理业
4,6,深振业A,2022-12-31,房地产业
...,...,...,...,...
5142,900929,锦旅B股,2022-12-31,商务服务业
5143,900939,汇丽B,2022-12-31,建筑装饰和其他建筑业
5144,900948,伊泰B股,2022-12-31,煤炭开采和洗选业
5145,900953,凯马B,2022-12-31,汽车制造业


In [14]:
agg_df['industry'] = ''
for i in range(len(agg_df)):
    try:
        ind = industry.loc[industry['Symbol']==agg_df.loc[i,'Stkcd']]['IndustryName'].values[0]
        agg_df.loc[i,'industry'] = ind
    except:
        pass

agg_df就是合并后的数据集合

In [15]:
agg_df

,Stkcd,Trddt,Clsprc,return,PB,Maeket Value,industry
0,1,2022-11-01,10.670,,0.476355,5142219313576.772461,货币金融服务
1,1,2022-11-02,10.630,-0.003749,0.474569,5142219313576.772461,货币金融服务
2,1,2022-11-03,10.440,-0.017874,0.466087,5142219313576.772461,货币金融服务
3,1,2022-11-04,10.820,0.036398,0.483052,5142219313576.772461,货币金融服务
4,1,2022-11-07,10.880,0.005545,0.48573,5142219313576.772461,货币金融服务
...,...,...,...,...,...,...,...
1271139,900957,2023-10-25,0.356,0.017143,0.207639,1275026745.62298,房地产业
1271140,900957,2023-10-26,0.364,0.022472,0.212305,1275026745.62298,房地产业
1271141,900957,2023-10-27,0.370,0.016484,0.215804,1275026745.62298,房地产业
1271142,900957,2023-10-30,0.393,0.062162,0.229219,1275026745.62298,房地产业


In [16]:
# v.计算Rm-Rf,SMB,HML三因子
free_rate = pd.read_excel(r'C:\Users\62589\Desktop\MBK_SHIBORM.xlsx')
free_rate

,SgnDate,Term,Shibor
0,2022-11-01,6个月,1.884
1,2022-11-01,1天,1.843
2,2022-11-01,1年,2.038
3,2022-11-01,9个月,1.980
4,2022-11-01,14天,1.652
...,...,...,...
1995,2023-10-31,7天,1.965
1996,2023-10-31,9个月,2.504
1997,2023-10-31,1年,2.530
1998,2023-10-31,6个月,2.475
